In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# VGG16

In [2]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import Model

# Data preprocessing
train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=20, zoom_range=0.2, horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/ML/ML_DATA/train',
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

validation_generator = test_datagen.flow_from_directory(
    '/content/drive/MyDrive/ML/ML_DATA/test',
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

# Calculate steps per epoch and validation steps
steps_per_epoch = (5250 // 32) + 1
validation_steps = (624 // 32) + 1

# Load the VGG16 model, excluding the top layers
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(150, 150, 3))

# Freeze the base model
for layer in base_model.layers:
    layer.trainable = False

# Add custom layers on top of VGG16
model = Sequential([
    base_model,
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=validation_steps
)

# Save the model
model.save('my_vgg16_binary_classification_model.h5')

Found 5250 images belonging to 2 classes.
Found 624 images belonging to 2 classes.
58889256/58889256 [==============================] - 0s 0us/step
Epoch 1/10
165/165 [==============================] - 717s 4s/step - loss: 0.3082 - accuracy: 0.8859 - val_loss: 0.2173 - val_accuracy: 0.9231
Epoch 2/10
165/165 [==============================] - 110s 664ms/step - loss: 0.1793 - accuracy: 0.9312 - val_loss: 0.1944 - val_accuracy: 0.9263
Epoch 3/10
165/165 [==============================] - 110s 665ms/step - loss: 0.1716 - accuracy: 0.9343 - val_loss: 0.1746 - val_accuracy: 0.9359
Epoch 4/10
165/165 [==============================] - 110s 668ms/step - loss: 0.1691 - accuracy: 0.9333 - val_loss: 0.2544 - val_accuracy: 0.8894
Epoch 5/10
165/165 [==============================] - 116s 704ms/step - loss: 0.1568 - accuracy: 0.9440 - val_loss: 0.1713 - val_accuracy: 0.9311
Epoch 6/10
165/165 [==============================] - 106s 641ms/step - loss: 0.1480 - accuracy: 0.9429 - val_loss: 0.1791 - 

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [3]:
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# Load the saved model
model = load_model('my_vgg16_binary_classification_model.h5')

# Data preprocessing
test_datagen = ImageDataGenerator(rescale=1./255)

# Load test data
test_generator = test_datagen.flow_from_directory(
    '/content/drive/MyDrive/ML/ML_DATA/test',
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary',
    shuffle=False  # Important to keep the order for calculating metrics
)

# Evaluate the model on test data
y_pred_proba = model.predict(test_generator)
y_pred = (y_pred_proba > 0.5).astype(int)  # Convert probabilities to binary predictions

# True labels
y_true = test_generator.classes

# Calculate evaluation metrics
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)
roc_auc = roc_auc_score(y_true, y_pred_proba)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("ROC-AUC Score:", roc_auc)


Found 624 images belonging to 2 classes.
20/20 [==============================] - 9s 431ms/step
Accuracy: 0.9407051282051282
Precision: 0.9379652605459057
Recall: 0.9692307692307692
F1 Score: 0.9533417402269861
ROC-AUC Score: 0.9850646504492658


# CNN

In [4]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Data preprocessing
train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=20, zoom_range=0.2, horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/ML/ML_DATA/train',
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

validation_generator = test_datagen.flow_from_directory(
    '/content/drive/MyDrive/ML/ML_DATA/test',
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

# Calculate steps per epoch and validation steps
steps_per_epoch = (5250 // 32) + 1
validation_steps = (624 // 32) + 1

# Model architecture
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=validation_steps
)

# Save the model
model.save('my_binary_classification_model.h5')

Found 5250 images belonging to 2 classes.
Found 624 images belonging to 2 classes.
Epoch 1/10
165/165 [==============================] - 146s 640ms/step - loss: 0.4236 - accuracy: 0.8166 - val_loss: 0.2580 - val_accuracy: 0.8974
Epoch 2/10
165/165 [==============================] - 105s 636ms/step - loss: 0.2464 - accuracy: 0.9011 - val_loss: 0.2894 - val_accuracy: 0.8702
Epoch 3/10
165/165 [==============================] - 102s 618ms/step - loss: 0.2119 - accuracy: 0.9168 - val_loss: 0.3670 - val_accuracy: 0.8253
Epoch 4/10
165/165 [==============================] - 103s 623ms/step - loss: 0.2053 - accuracy: 0.9223 - val_loss: 0.2372 - val_accuracy: 0.8910
Epoch 5/10
165/165 [==============================] - 109s 664ms/step - loss: 0.1739 - accuracy: 0.9303 - val_loss: 0.1543 - val_accuracy: 0.9599
Epoch 6/10
165/165 [==============================] - 101s 611ms/step - loss: 0.1761 - accuracy: 0.9307 - val_loss: 0.2353 - val_accuracy: 0.8926
Epoch 7/10
165/165 [=====================

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [5]:
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# Load the saved model
model = load_model('my_binary_classification_model.h5')

# Data preprocessing
test_datagen = ImageDataGenerator(rescale=1./255)

# Load test data
test_generator = test_datagen.flow_from_directory(
    '/content/drive/MyDrive/ML/ML_DATA/test',
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary',
    shuffle=False  # Important to keep the order for calculating metrics
)

# Evaluate the model on test data
y_pred_proba = model.predict(test_generator)
y_pred = (y_pred_proba > 0.5).astype(int)  # Convert probabilities to binary predictions

# True labels
y_true = test_generator.classes

# Calculate evaluation metrics
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)
roc_auc = roc_auc_score(y_true, y_pred_proba)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("ROC-AUC Score:", roc_auc)


Found 624 images belonging to 2 classes.
20/20 [==============================] - 8s 382ms/step
Accuracy: 0.9647435897435898
Precision: 0.9577114427860697
Recall: 0.9871794871794872
F1 Score: 0.9722222222222222
ROC-AUC Score: 0.9877492877492877


# ResNet-101

In [7]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet101
from tensorflow.keras.optimizers import Adam

# Data preprocessing
train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=20, zoom_range=0.2, horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/ML/ML_DATA/train',
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

validation_generator = test_datagen.flow_from_directory(
    '/content/drive/MyDrive/ML/ML_DATA/test',
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

# Calculate steps per epoch and validation steps
steps_per_epoch = (5250 // 32) + 1
validation_steps = (624 // 32) + 1

# Load ResNet-101 with pre-trained weights and without the top layer
base_model = ResNet101(weights='imagenet', include_top=False, input_shape=(150, 150, 3))

# Freeze the base model
base_model.trainable = False

# Add new layers on top of the base model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(1, activation='sigmoid')(x)

# Construct the final model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=validation_steps
)

# Save the model
model.save('my_binary_classification_model_resnet101.h5')

Found 5250 images belonging to 2 classes.
Found 624 images belonging to 2 classes.
171446536/171446536 [==============================] - 1s 0us/step
Epoch 1/10
165/165 [==============================] - 160s 657ms/step - loss: 0.5895 - accuracy: 0.7196 - val_loss: 0.6564 - val_accuracy: 0.6250
Epoch 2/10
165/165 [==============================] - 105s 638ms/step - loss: 0.5411 - accuracy: 0.7419 - val_loss: 0.6047 - val_accuracy: 0.6378
Epoch 3/10
165/165 [==============================] - 105s 638ms/step - loss: 0.5153 - accuracy: 0.7470 - val_loss: 0.5853 - val_accuracy: 0.6571
Epoch 4/10
165/165 [==============================] - 111s 671ms/step - loss: 0.5009 - accuracy: 0.7528 - val_loss: 0.5585 - val_accuracy: 0.6779
Epoch 5/10
165/165 [==============================] - 105s 633ms/step - loss: 0.4834 - accuracy: 0.7617 - val_loss: 0.5528 - val_accuracy: 0.6859
Epoch 6/10
165/165 [==============================] - 114s 688ms/step - loss: 0.4656 - accuracy: 0.7697 - val_loss: 0.52

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [8]:
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# Load the saved model
model = load_model('my_binary_classification_model_resnet101.h5')

# Data preprocessing
test_datagen = ImageDataGenerator(rescale=1./255)

# Load test data
test_generator = test_datagen.flow_from_directory(
    '/content/drive/MyDrive/ML/ML_DATA/test',
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary',
    shuffle=False  # Important to keep the order for calculating metrics
)

# Evaluate the model on test data
y_pred_proba = model.predict(test_generator)
y_pred = (y_pred_proba > 0.5).astype(int)  # Convert probabilities to binary predictions

# True labels
y_true = test_generator.classes

# Calculate evaluation metrics
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)
roc_auc = roc_auc_score(y_true, y_pred_proba)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("ROC-AUC Score:", roc_auc)


Found 624 images belonging to 2 classes.
20/20 [==============================] - 11s 436ms/step
Accuracy: 0.7147435897435898
Precision: 0.6955719557195572
Recall: 0.9666666666666667
F1 Score: 0.8090128755364808
ROC-AUC Score: 0.871356563664256


# Denese Net 121

In [9]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Data preprocessing
train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=20, zoom_range=0.2, horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/ML/ML_DATA/train',
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

validation_generator = test_datagen.flow_from_directory(
    '/content/drive/MyDrive/ML/ML_DATA/test',
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

# Calculate steps per epoch and validation steps
steps_per_epoch = (5250 // 32) + 1
validation_steps = (624 // 32) + 1

# Load the DenseNet121 model, excluding the top layers
base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(150, 150, 3))

# Freeze the base model
for layer in base_model.layers:
    layer.trainable = False

# Add custom layers on top of DenseNet121
model = Sequential([
    base_model,
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=validation_steps
)

# Save the model
model.save('my_densenet121_binary_classification_model.h5')


Found 5250 images belonging to 2 classes.
Found 624 images belonging to 2 classes.
29084464/29084464 [==============================] - 0s 0us/step
Epoch 1/10
165/165 [==============================] - 129s 660ms/step - loss: 0.8952 - accuracy: 0.8775 - val_loss: 0.3791 - val_accuracy: 0.8301
Epoch 2/10
165/165 [==============================] - 112s 679ms/step - loss: 0.1916 - accuracy: 0.9213 - val_loss: 0.2214 - val_accuracy: 0.9087
Epoch 3/10
165/165 [==============================] - 111s 675ms/step - loss: 0.1882 - accuracy: 0.9301 - val_loss: 0.1992 - val_accuracy: 0.9183
Epoch 4/10
165/165 [==============================] - 103s 622ms/step - loss: 0.1885 - accuracy: 0.9259 - val_loss: 0.1967 - val_accuracy: 0.9167
Epoch 5/10
165/165 [==============================] - 108s 657ms/step - loss: 0.1856 - accuracy: 0.9242 - val_loss: 0.1817 - val_accuracy: 0.9247
Epoch 6/10
165/165 [==============================] - 113s 685ms/step - loss: 0.1689 - accuracy: 0.9373 - val_loss: 0.1964

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [10]:
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# Load the saved model
model = load_model('my_densenet121_binary_classification_model.h5')

# Data preprocessing
test_datagen = ImageDataGenerator(rescale=1./255)

# Load test data
test_generator = test_datagen.flow_from_directory(
    '/content/drive/MyDrive/ML/ML_DATA/test',
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary',
    shuffle=False  # Important to keep the order for calculating metrics
)

# Evaluate the model on test data
y_pred_proba = model.predict(test_generator)
y_pred = (y_pred_proba > 0.5).astype(int)  # Convert probabilities to binary predictions

# True labels
y_true = test_generator.classes

# Calculate evaluation metrics
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)
roc_auc = roc_auc_score(y_true, y_pred_proba)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("ROC-AUC Score:", roc_auc)


Found 624 images belonging to 2 classes.
20/20 [==============================] - 11s 439ms/step
Accuracy: 0.9358974358974359
Precision: 0.9310344827586207
Recall: 0.9692307692307692
F1 Score: 0.949748743718593
ROC-AUC Score: 0.9847797501643656


# mobile

In [12]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.optimizers import Adam

# Data preprocessing
train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=20, zoom_range=0.2, horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/ML/ML_DATA/train',
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

validation_generator = test_datagen.flow_from_directory(
    '/content/drive/MyDrive/ML/ML_DATA/test',
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

# Calculate steps per epoch and validation steps
steps_per_epoch = (5250 // 32) + 1
validation_steps = (624 // 32) + 1

# Load MobileNetV2 with pre-trained weights and without the top layer
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the base model
base_model.trainable = False

# Add new layers on top of the base model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(1, activation='sigmoid')(x)

# Construct the final model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=validation_steps
)

# Save the model
model.save('my_mobile_classification_model_mobilenetv2.h5')


Found 5250 images belonging to 2 classes.
Found 624 images belonging to 2 classes.
Epoch 1/10
165/165 [==============================] - 114s 669ms/step - loss: 0.3855 - accuracy: 0.8411 - val_loss: 0.3735 - val_accuracy: 0.8526
Epoch 2/10
165/165 [==============================] - 106s 641ms/step - loss: 0.2234 - accuracy: 0.9088 - val_loss: 0.2830 - val_accuracy: 0.8878
Epoch 3/10
165/165 [==============================] - 100s 608ms/step - loss: 0.1967 - accuracy: 0.9234 - val_loss: 0.2480 - val_accuracy: 0.9054
Epoch 4/10
165/165 [==============================] - 102s 615ms/step - loss: 0.1794 - accuracy: 0.9288 - val_loss: 0.2990 - val_accuracy: 0.8830
Epoch 5/10
165/165 [==============================] - 101s 614ms/step - loss: 0.1667 - accuracy: 0.9341 - val_loss: 0.2846 - val_accuracy: 0.8894
Epoch 6/10
165/165 [==============================] - 101s 609ms/step - loss: 0.1645 - accuracy: 0.9354 - val_loss: 0.2661 - val_accuracy: 0.8910
Epoch 7/10
165/165 [=====================

In [13]:
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# Load the saved model
model = load_model('my_mobile_classification_model_mobilenetv2.h5')

# Data preprocessing
test_datagen = ImageDataGenerator(rescale=1./255)

# Load test data
test_generator = test_datagen.flow_from_directory(
    '/content/drive/MyDrive/ML/ML_DATA/test',
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary',
    shuffle=False  # Important to keep the order for calculating metrics
)

# Evaluate the model on test data
y_pred_proba = model.predict(test_generator)
y_pred = (y_pred_proba > 0.5).astype(int)  # Convert probabilities to binary predictions

# True labels
y_true = test_generator.classes

# Calculate evaluation metrics
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)
roc_auc = roc_auc_score(y_true, y_pred_proba)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("ROC-AUC Score:", roc_auc)


Found 624 images belonging to 2 classes.
20/20 [==============================] - 9s 417ms/step
Accuracy: 0.8942307692307693
Precision: 0.8732718894009217
Recall: 0.9717948717948718
F1 Score: 0.9199029126213591
ROC-AUC Score: 0.9672802980495288


# GUI

In [14]:
!pip install --upgrade gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 57.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 13.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.9/315.9 kB 42.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 70.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 11.1 M

In [15]:
!pip install gradio

In [21]:
import gradio as gr
import tensorflow as tf
from tensorflow.keras.preprocessing import image
import numpy as np

# Define the available models and their paths
model_paths = {
    "CNN": "my_binary_classification_model.h5",
    "VGG16": "my_vgg16_binary_classification_model.h5",
    "DenseNet121 ": "my_densenet121_binary_classification_model.h5",
    "MobileNetV2":"my_mobile_classification_model_mobilenetv2.h5",
    "ResNet101":"my_binary_classification_model_resnet101.h5"
}

# Load a model based on the selected path
def load_model(model_name):
    return tf.keras.models.load_model(model_paths[model_name])

# Define the function to make predictions
def predict_pneumonia(model_name, img):
  if model_name =="MobileNetV2":
    model = load_model(model_name)  # Load the selected model
    img = img.resize((224, 224))  # Resize the image to the expected input size
    img_array = image.img_to_array(img)  # Convert image to array
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array /= 255.0  # Rescale the image
  else:
    model = load_model(model_name)  # Load the selected model
    img = img.resize((150, 150))  # Resize the image to the expected input size
    img_array = image.img_to_array(img)  # Convert image to array
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array /= 255.0  # Rescale the image

  prediction = model.predict(img_array)  # Make prediction
  if prediction[0] > 0.5:
     return "Pneumonia"
  else:
     return "Normal"

# Create Gradio interface
demo = gr.Interface(
    fn=predict_pneumonia,
    inputs=[
        gr.Dropdown(choices=list(model_paths.keys()), label="Select Model"),  # Dropdown for model selection
        gr.Image(type="pil", label="Upload X-ray Image")  # Image upload
    ],
    outputs=gr.Textbox(label="Prediction")  # Output textbox
)

# Launch the interface
demo.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://988f6cbc62aa3c574f.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
